<a href="https://colab.research.google.com/github/hankaj/Machine-Learning-Techniques/blob/master/ML2021_Second_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
url='https://raw.githubusercontent.com/hankaj/Machine-Learning-Techniques/master/comments.csv'

In [2]:
import pandas as pd
df1 = pd.read_csv(url)
df1

,class,text
0,Auto,I have recently purchased a J30T with moderat...
1,Camera,I bought this product because I need instant ...
2,Auto,I have owned my Buick since 53000 km and I am...
3,Camera,This was my first Digital camera so I did qui...
4,Camera,Minolta DiMAGE 7Hi is in a digital SLR with 5...
...,...,...
595,Auto,Recently our 12 year old Nissan Stanza decide...
596,Camera,I always do a lot of research before I buy an...
597,Auto,This car is an all around good buy If you ar...
598,Auto,I waited to write this until I have had 4 mon...


In [3]:
y = df1.iloc[:,0]
y

0        Auto
1      Camera
2        Auto
3      Camera
4      Camera
        ...  
595      Auto
596    Camera
597      Auto
598      Auto
599      Auto
Name: class, Length: 600, dtype: object

In [4]:
X = df1.iloc[:, 1]
X

0       I have recently purchased a J30T with moderat...
1       I bought this product because I need instant ...
2       I have owned my Buick since 53000 km and I am...
3       This was my first Digital camera so I did qui...
4       Minolta DiMAGE 7Hi is in a digital SLR with 5...
                             ...                        
595     Recently our 12 year old Nissan Stanza decide...
596     I always do a lot of research before I buy an...
597     This car is an all around good buy  If you ar...
598     I waited to write this until I have had 4 mon...
599     I have been a Montero owner since about 1985 ...
Name: text, Length: 600, dtype: object

In [5]:
import re
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize, sent_tokenize

new_features = pd.DataFrame(np.zeros((X.shape[0], 2)), columns=['pos', 'number of words'])

for com_idx, comment in enumerate(X):
  # remove useless data: !"_$%&/()=_ˆ*¡@
  comment = re.sub('[!@#$%^&*()_+<>]', '', comment)

  # convert to lowercase
  comment = comment.lower()

  # substitute multiple spaces with single space
  comment = re.sub(r'\s+', ' ', comment, flags=re.I)

  # lemmatize and get features
  tokens = word_tokenize(comment)
  n_words = len(tokens)
  comment = [lemmatizer.lemmatize(t, pos = 'v') for t in tokens]
  tags = nltk.pos_tag(tokens)
  comment = ' '.join(comment)
  tags = ' '.join([word[1] for word in tags])

  X[com_idx] = comment
  new_features.iloc[com_idx, 0] = tags
  new_features.iloc[com_idx, 1] = n_words

X=pd.concat([X, new_features], axis=1)
X

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,text,pos,number of words
0,i have recently purchase a j30t with moderate ...,NNS VBP RB VBN DT NN IN JJ NNS RB VBD IN DT NN...,221.0
1,i buy this product because i need instant grat...,NN VBD DT NN IN JJ VBP JJ NN NN MD VB VB DT NN...,536.0
2,i have own my buick since 53000 km and i be no...,NNS VBP VBN PRP$ NN IN CD NN CC VB VBP RB VBG ...,278.0
3,this be my first digital camera so i do quite ...,DT VBD PRP$ JJ JJ NN IN JJ VBD RB DT NN IN NN ...,202.0
4,minolta dimage 7hi be in a digital slr with 5 ...,JJ NN CD VBZ IN DT JJ NN IN CD NNS CD CD NN NN...,974.0
...,...,...,...
595,recently our 12 year old nissan stanza decide ...,RB PRP$ CD NN JJ JJ NN VBD PRP VBD NN TO VB RB...,408.0
596,i always do a lot of research before i buy any...,NN RB VBP DT NN IN NN IN JJ VBP NN RB PRP VBP ...,341.0
597,this car be an all around good buy if you be i...,DT NN VBZ DT RB RB JJ NN IN PRP VBP IN NNS PRP...,158.0
598,i wait to write this until i have have 4 month...,RB VBD TO VB DT IN NNS VBP VBD CD NNS IN VBG P...,461.0


In [6]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3)
X_train

,text,pos,number of words
134,i get this camera for about 50 on ebay it reta...,NN VBD DT NN IN RB CD IN NN PRP VBZ IN IN CD C...,228.0
443,i have own a 1990 subaru legacy awd for 6 year...,NN VBD VBN DT CD NN NN NN IN CD NNS CC VBD RB ...,294.0
150,my wife and i be look for a 7 or more passenge...,PRP$ NN CC NN VBD VBG IN DT CD CC JJR NN NN CC...,301.0
1,i buy this product because i need instant grat...,NN VBD DT NN IN JJ VBP JJ NN NN MD VB VB DT NN...,536.0
371,about a year ago i purchase a 775 from dell 90...,IN DT NN RB NN VBD DT CD IN NN CD NNS RB VBP N...,371.0
...,...,...,...
389,the dodge viper be in a class all by itself wi...,DT NN NN VBZ IN DT NN DT IN PRP IN PRP$ NN NN ...,282.0
29,if you put on tons of miles but want to look g...,IN PRP VBP IN NNS IN NNS CC VBP TO VB JJ VBG P...,199.0
156,i be not new to digital cameras and in a way i...,NN VBP RB JJ TO JJ NNS CC IN DT NN NN VBP RB V...,1573.0
374,be use the canon digital elph s110 powershot a...,VBN VBG DT NN JJ NN NN NN IN VBG VBN DT NN VBD...,695.0


In [7]:
# vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

#TFIDF
vectorizer = TfidfVectorizer()
vectorizer.fit(X.iloc[:, 0])

X_train_Tfidf = vectorizer.transform(X_train.iloc[:, 0])
X_test_Tfidf = vectorizer.transform(X_test.iloc[:, 0])

In [8]:
#TFIDF + N-grams
vectorizer_NGrams = TfidfVectorizer(ngram_range=(1, 2))
vectorizer_NGrams.fit(X['text'])

X_train_Tfidf_NGrams = vectorizer.transform(X_train['text'])
X_test_Tfidf_NGrams = vectorizer.transform(X_test['text'])

In [24]:
#TFIDF + N-grams + POS tagging
import scipy.sparse as sp

vectorizerPOS = TfidfVectorizer(ngram_range=(1, 2))
vectorizerPOS.fit(X['pos'])
X_train_POS = vectorizerPOS.transform(X_train['pos'])
X_test_POS = vectorizerPOS.transform(X_test['pos'])

X_train_Tfidf_NGrams_POS =  sp.hstack([X_train_Tfidf_NGrams, X_train_POS], format='csr')
X_test_Tfidf_NGrams_POS =  sp.hstack([X_test_Tfidf_NGrams, X_test_POS], format='csr')


(420, 11312)
(420, 3)


In [48]:
#TFIDF + N-grams + POS tagging + number of words
X_train_other = pd.DataFrame.sparse.from_spmatrix(X_train_Tfidf_NGrams_POS)
X_test_other = pd.DataFrame.sparse.from_spmatrix(X_test_Tfidf_NGrams_POS)
X_train_other['number of words'] = X_train['number of words']
X_test_other['number of words'] = X_test['number of words']

In [54]:
from sklearn import svm
SVM = svm.SVC(kernel='linear')
SVM.fit(X_train_Tfidf, y_train)

y_pred_Tfidf = SVM.predict(X_test_Tfidf)

In [15]:
SVM.fit(X_train_Tfidf_NGrams, y_train)
y_pred_Tfidf_Ngrams = SVM.predict(X_test_Tfidf_NGrams)

In [16]:
SVM.fit(X_train_Tfidf_NGrams_POS, y_train)
y_pred_Tfidf_Ngrams_POS = SVM.predict(X_test_Tfidf_NGrams_POS)

In [50]:
X_train_other.fillna(999, inplace=True)
X_test_other.fillna(999, inplace=True)
SVM.fit(X_train_other, y_train)
y_pred_other = SVM.predict(X_test_other)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


In [52]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#TFIDF
print(confusion_matrix(y_test,y_pred_Tfidf))
print(classification_report(y_test,y_pred_Tfidf))
print(accuracy_score(y_test, y_pred_Tfidf))

[[ 78   0]
 [  0 102]]
              precision    recall  f1-score   support

        Auto       1.00      1.00      1.00        78
      Camera       1.00      1.00      1.00       102

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180

1.0


In [57]:
#TFIDF + N-grams
print(confusion_matrix(y_test,y_pred_Tfidf_Ngrams))
print(classification_report(y_test,y_pred_Tfidf_Ngrams))
print(accuracy_score(y_test, y_pred_Tfidf_Ngrams))

[[ 78   0]
 [  0 102]]
              precision    recall  f1-score   support

        Auto       1.00      1.00      1.00        78
      Camera       1.00      1.00      1.00       102

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180

1.0


In [58]:
#TFIDF + N-grams + POS tagging
print(confusion_matrix(y_test,y_pred_Tfidf_Ngrams_POS))
print(classification_report(y_test,y_pred_Tfidf_Ngrams_POS))
print(accuracy_score(y_test, y_pred_Tfidf_Ngrams_POS))

[[ 78   0]
 [  0 102]]
              precision    recall  f1-score   support

        Auto       1.00      1.00      1.00        78
      Camera       1.00      1.00      1.00       102

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180

1.0


In [59]:
#TFIDF + N-grams + POS tagging + number of words
print(confusion_matrix(y_test,y_pred_other))
print(classification_report(y_test,y_pred_other))
print(accuracy_score(y_test, y_pred_other))

[[ 78   0]
 [  0 102]]
              precision    recall  f1-score   support

        Auto       1.00      1.00      1.00        78
      Camera       1.00      1.00      1.00       102

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180

1.0


In [61]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train_Tfidf, y_train)
y_pred_Tfidf = classifier.predict(X_test_Tfidf)

In [62]:
classifier.fit(X_train_Tfidf_NGrams, y_train)
y_pred_Tfidf_NGrams = classifier.predict(X_test_Tfidf_NGrams)

In [63]:
classifier.fit(X_train_Tfidf_NGrams_POS, y_train)
y_pred_Tfidf_NGrams_POS = classifier.predict(X_test_Tfidf_NGrams_POS)

In [64]:
classifier.fit(X_train_other, y_train)
y_pred_other = classifier.predict(X_test_other)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:625: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  "pandas.DataFrame with sparse columns found."


In [65]:
#TFIDF
print(confusion_matrix(y_test,y_pred_Tfidf))
print(classification_report(y_test,y_pred_Tfidf))
print(accuracy_score(y_test, y_pred_Tfidf))

[[ 78   0]
 [  0 102]]
              precision    recall  f1-score   support

        Auto       1.00      1.00      1.00        78
      Camera       1.00      1.00      1.00       102

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180

1.0


In [66]:
#TFIDF + N-grams
print(confusion_matrix(y_test,y_pred_Tfidf_Ngrams))
print(classification_report(y_test,y_pred_Tfidf_Ngrams))
print(accuracy_score(y_test, y_pred_Tfidf_Ngrams))

[[ 78   0]
 [  0 102]]
              precision    recall  f1-score   support

        Auto       1.00      1.00      1.00        78
      Camera       1.00      1.00      1.00       102

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180

1.0


In [67]:
#TFIDF + N-grams + POS tagging
print(confusion_matrix(y_test,y_pred_Tfidf_Ngrams_POS))
print(classification_report(y_test,y_pred_Tfidf_Ngrams_POS))
print(accuracy_score(y_test, y_pred_Tfidf_Ngrams_POS))

[[ 78   0]
 [  0 102]]
              precision    recall  f1-score   support

        Auto       1.00      1.00      1.00        78
      Camera       1.00      1.00      1.00       102

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180

1.0


In [68]:
#TFIDF + N-grams + POS tagging + number of words
print(confusion_matrix(y_test,y_pred_other))
print(classification_report(y_test,y_pred_other))
print(accuracy_score(y_test, y_pred_other))

[[ 78   0]
 [  0 102]]
              precision    recall  f1-score   support

        Auto       1.00      1.00      1.00        78
      Camera       1.00      1.00      1.00       102

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180

1.0
